In [1]:
# !pip install -U bitsandbytes --user

In [2]:
!nvidia-smi

Tue Jun 24 17:38:31 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          Off | 00000000:04:00.0 Off |                    0 |
| N/A   34C    P0              68W / 700W |      3MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset

/opt/conda/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-24 17:38:35.549705: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-24 17:38:35.560715: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750786715.574206   44103 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750786715.578060   44103 cuda_blas.cc:1407] Unable t

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

[2025-06-24 17:38:37,417] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/envs/py311/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/conda/envs/py311/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/conda/envs/py311/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/opt/conda/envs/py311/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/opt/conda/envs/py311/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/opt/conda/envs/py311/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::chrono::_V2::steady_clock::now()@GLIBCXX_3.4.19'
/opt/conda/envs/py311/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::string::_M_replace_aux(uns

[2025-06-24 17:38:38,543] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


In [4]:
dataset = load_dataset("json", data_files = "/projects/neural-alchemists-ftf-hackathon/neural-alchemists-ftf-hackathon/task_trajectories_v3.jsonl", split="train")

In [5]:
model_path = '/projects/llm-repo/models/Qwen/Qwen2.5-14B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto"
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 8/8 [00:23<00:00,  2.91s/it]


In [8]:
model = prepare_model_for_kbit_training(model)

In [9]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model,lora_config)

In [10]:
model.print_trainable_parameters()

trainable params: 68,812,800 || all params: 14,838,846,464 || trainable%: 0.4637


In [11]:
training_args = TrainingArguments(
    output_dir="./qwen_venmo_sft_14b_v2",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    save_steps=500,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    lr_scheduler_type='cosine',
    report_to="none",
)

In [12]:
trainer = SFTTrainer(
    model = model,
    train_dataset=dataset,
    peft_config=lora_config,
    processing_class=tokenizer,
    args=training_args
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/envs/py311/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.833400
20,1.437600
30,0.916700
40,0.569900
50,0.170500
60,0.053500
70,0.038700
80,0.041300
90,0.039600
100,0.039400


/opt/conda/envs/py311/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/envs/py311/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/envs/py311/lib/python

TrainOutput(global_step=4375, training_loss=0.043548046585491725, metrics={'train_runtime': 28000.2448, 'train_samples_per_second': 2.5, 'train_steps_per_second': 0.156, 'total_flos': 5.095994057087558e+18, 'train_loss': 0.043548046585491725})

In [14]:
trainer.model.save_pretrained('./qwen_finetuned_14b_v2')
tokenizer.save_pretrained('./qwen_finetuned_14b_v2')

('./qwen_finetuned_14b_v1/tokenizer_config.json',
 './qwen_finetuned_14b_v1/special_tokens_map.json',
 './qwen_finetuned_14b_v1/vocab.json',
 './qwen_finetuned_14b_v1/merges.txt',
 './qwen_finetuned_14b_v1/added_tokens.json',
 './qwen_finetuned_14b_v1/tokenizer.json')

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Step 1: Load the base model with standard HuggingFace class
base_model = AutoModelForCausalLM.from_pretrained(
    "/projects/llm-repo/models/Qwen/Qwen2.5-14B-Instruct",
    torch_dtype="auto",
    device_map="auto"  # Optional: only if needed
)

# Step 2: Load and merge LoRA adapter
model = PeftModel.from_pretrained(base_model, "./qwen_finetuned_14b_v2")
model = model.merge_and_unload()

# Step 3: Save merged model
model.save_pretrained("merged_qwen25_14B_v2")
tokenizer = AutoTokenizer.from_pretrained("/projects/llm-repo/models/Qwen/Qwen2.5-14B-Instruct")
tokenizer.save_pretrained("merged_qwen25_14B_v2")

Loading checkpoint shards: 100%|██████████| 8/8 [00:03<00:00,  2.35it/s]
/opt/conda/envs/py311/lib/python3.11/site-packages/transformers/modeling_utils.py:3391: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(
Saving checkpoint shards: 100%|██████████| 6/6 [01:02<00:00, 10.40s/it]


('merged_qwen25_14B_v2/tokenizer_config.json',
 'merged_qwen25_14B_v2/special_tokens_map.json',
 'merged_qwen25_14B_v2/vocab.json',
 'merged_qwen25_14B_v2/merges.txt',
 'merged_qwen25_14B_v2/added_tokens.json',
 'merged_qwen25_14B_v2/tokenizer.json')